### backend

In [1]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=800,  seed_transpiler=seed, seed_simulator=seed)

### optimizer

In [2]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

In [15]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

In [16]:
import numpy as np
from qiskit.circuit.library import EfficientSU2

num_qubits = 2
entanglement = 'full'
energies = np.zeros(2)

ansatz = EfficientSU2(num_qubits=num_qubits, entanglement=entanglement)

In [20]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

params = np.random.rand(3)
print(get_var_form(params))

ret = bfgs_optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)
print(ret)
print(get_var_form(ret[0]))

C:\Users\hadar\AppData\Local\Temp\ipykernel_36308\2317275169.py:6: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  qc.u3(params[0], params[1], params[2], qr[0])


     ┌──────────────────────────────┐┌─┐
  q: ┤ U3(0.14172,0.094188,0.91319) ├┤M├
     └──────────────────────────────┘└╥┘
c: 1/═════════════════════════════════╩═
                                      0 
(array([0.14172334, 0.09418886, 0.91318713]), 0.959859881261161, 88)
     ┌──────────────────────────────┐┌─┐
  q: ┤ U3(0.14172,0.094189,0.91319) ├┤M├
     └──────────────────────────────┘└╥┘
c: 1/═════════════════════════════════╩═
                                      0 


In [13]:
from qiskit import Aer, transpile, assemble
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    
    return cost

In [ ]:
from collections.abc import Sequence

import numpy as np

class ExpvalCost:
    """
    Create a cost function that gives the expectation value of an input Hamiltonian.

    This cost function is useful for a range of problems including VQE and QAOA.
    """

    def __init__(
        self,
        ansatz,
        hamiltonian,
        device,
    ):
        self.ansatz = ansatz
        
        coeffs, observables = hamiltonian.terms()

        self.hamiltonian = hamiltonian

        self._optimize = optimize

        self.qnodes = qml.map(
            ansatz, observables, device, interface=interface, diff_method=diff_method, **kwargs
        )

        if self._optimize:
            obs_groupings, coeffs_groupings = qml.grouping.group_observables(observables, coeffs)
            d = device[0] if self._multiple_devices else device
            w = d.wires.tolist()

            @qml.qnode(device, interface=interface, diff_method=diff_method, **kwargs)
            def circuit(*qnode_args, obs, **qnode_kwargs):
                """Converting ansatz into a full circuit including measurements"""
                ansatz(*qnode_args, wires=w, **qnode_kwargs)
                return [qml.expval(o) for o in obs]

            def cost_fn(*qnode_args, **qnode_kwargs):
                """Combine results from grouped QNode executions with grouped coefficients"""
                if device.shot_vector:
                    shots_batch = 0

                    for i in device.shot_vector:
                        shots_batch += i[1]

                    total = [0] * shots_batch

                    for o, c in zip(obs_groupings, coeffs_groupings):
                        res = circuit(*qnode_args, obs=o, **qnode_kwargs)

                        for i, batch_res in enumerate(res):
                            total[i] += sum(batch_res * c)
                else:
                    total = 0
                    for o, c in zip(obs_groupings, coeffs_groupings):
                        res = circuit(*qnode_args, obs=o, **qnode_kwargs)
                        total += sum([r * c_ for r, c_ in zip(res, c)])
                return total

            self.cost_fn = cost_fn

        else:
            self.cost_fn = qml.dot(coeffs, self.qnodes)


    def __call__(self, *params, state_idx):
        return self.cost_fn(*params, state_idx)


In [10]:
def single_cost(parameters):
    

[2 1]
